<a href="https://colab.research.google.com/github/petrtsv/aipoetry/blob/main/markov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install markovify

In [2]:
import markovify
from markovify.chain import BEGIN, END
import pickle
import requests
from collections import defaultdict
import random
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import sys
import json

sys.setrecursionlimit(1000000)

In [3]:
def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [4]:
def save_json_obj(obj, name):
  with open(name+'.json', 'w+') as f:
    json.dump(obj, f)

def load_json_obj(name):
  with open(name+'.json', 'r+') as f:
    return json.load(f)

In [5]:
def get_choices(model, state):
        """
        Given a state, give the list of next words and weights of this words.
        """
        try:
          choices = [choice for choice in model.chain.model[state].keys()]
          return choices
        except:
          return []

In [6]:
def get_rhymes(word):
  url = 'https://rifmus.net/rifma/' + word
  res = requests.get(url)
  soup = BeautifulSoup(res.text)
  words_table = soup.find('ul', {'class' : 'multicolumn'})
  result = []
  for ch in words_table.children:
    result.append(ch.string)
  result = set(result)
  result.remove('\n')
  return list(result)

def generate_rhyme_dict(text, filename):
  rhyme = load_obj(filename)
  res = {}
  words_set = set(text.split())
  words = list(words_set)
  for w in tqdm(words):
    st = rhyme[w]
    if st == {} or st & words_set == {}:
      res[w] = []
      continue
    res[w] = list(st & words_set)
  return res

def is_rhyme(a, b, rhyme_dict):
  try:
    return a in rhyme_dict[b] or b in rhyme_dict[a]
  except:
    return False

In [7]:
def generate_sentence(model, begin_state, tries = 100, min_cnt = 4, max_cnt = 6):
  result = ['default']
  length = random.randint(min_cnt, max_cnt)
  for i in range(tries):
    state = begin_state
    res = []
    if begin_state[1] != BEGIN and begin_state[1] != '$':
      res = [begin_state[1]]
    # print(length)
    for i in range(length):
      try:
        next = model.chain.move(state)
        if next == END:
          break
        # print(next)
        res.append(next)
      except:
        break
      state = (state[1], res[-1])
    if len(result) < min_cnt and len(res) > len(result):
      result = res.copy()
  result = ' '.join(result[::-1])
  return result

In [8]:
def generate_rhyme_line(model, prev_line, rhyme_dict):
  if prev_line == '':
    return ' '.join(generate_sentence(model, (BEGIN, '$'))[::-1])
  prev_word = prev_line.split()[-1]
  choices = get_choices(model, (BEGIN, '$'))
  rhyme_choices = [word for word in choices if is_rhyme(word, prev_word, rhyme_dict)]
  if rhyme_choices == []:
    print('havent choices')
    rhyme_choices = get_rhymes(prev_word)
  rhyme_word = rhyme_choices[random.randint(0, len(rhyme_choices)-1)]
  result = generate_sentence(model, ('$', rhyme_word))
  return result

In [9]:
text = ""
with open('sample_data/filtred_lines.txt') as f:
  text = f.read()

In [10]:
model = markovify.NewlineText(text, retain_original=1)

In [11]:
rhyme_dict = load_json_obj('rhyme_dict')

In [12]:
generate_sentence(model, (BEGIN, '$'))

'а меня не простит'

In [13]:
for i in range(20):
  fword = random.choice(list(rhyme_dict.keys()))
  first_line = generate_sentence(model, ('$', fword))
  second_line = generate_rhyme_line(model, first_line, rhyme_dict)
  print(first_line)
  print(second_line)


мы пара тварей с ноева ковчега
дева света где ты бегал
она мертва среди родного стана
знаю зачем я не отстану
havent choices
но вс таинственней и дивней
default
зарастает в груди забытых статуй
громкие галки над ним лохматый
рассыпался вдруг на золотой галере
как невесту лелеял как сына берег
havent choices
но тайна слов тех не разгадана
default
твердыни и от чего
звон и в лаврах сына своего
резва свободна и смела
в червонную землю давненько вросла
на мне платочков триста штук
ты работа нежных рук
в вечерний час звуча рассветом
это не ходячая монета
зажег везде лампады и свечи
молчальниц на вид такой приветный вечер
прелестный цвет любви увял
эпоха наша вся девятый вал
замке высоком никто не приходит
пафос мне не свойствен по природе
на тропинке с клюшкою железной
свершающих свой подвиг бесполезный
ровно половину тех солдат
по праву все безгрешные корят
грачи в чаду затрепетала
красавица очам твоим предстала
и ты как свеча
на фабрике немецкой вот сейчас
сын роскоши прохлад и нег
в злат